**For Fine-tuning**
===
> -*- coding: utf-8 -*-

> Author: xinghd
> Author: liudongliang


## Import libraries

In [1]:
from utils.utils import Trainer, Predictor, TrainReader,TestReader,PredictReader, data_collate, get_shuffle_data, collate_fn, seed_everything
from model.model import Encoder, Decoder, ModelCat

%reload_ext autoreload
%autoreload 2

In [40]:
import os
import json
import random
import time
import timeit
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from torch import nn
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import torch
from torch.utils.data import DataLoader, random_split
from utils.word2vec import seq_to_kmers, get_protein_embedding
from gensim.models import Word2Vec
from torch.utils import data as torch_data
import warnings

### Parameters

In [41]:
assert torch.cuda.is_available(), "Must have a avaliable gpu"

class CFG :
    # train dataset and test dataset
    traindata_path = './fine_tuning/train_cold_protein.csv'
    testdata_path = './fine_tuning/test_cold_protein.csv'
    # epoch
    EPOCHES = 20
    # batch size
    BATCH_SIZE = 1
    
    # learning rate
    lr = 1e-5
    # weight_decay
    weight_decay = 1e-7
    
    # suffix of log/model filename
    modelsave_file_suffix = 'funetuning_cold_protein.pt'
    result_file_suffix = 'funetuning_log_cold_protein.txt'
    
    # create log file
    result_file_path = './results/log/funetuning_log/'
    # save_best_model(depends on AUC)
    best_model_savepath = "./model/model_funetuning_cold_protein/"

    
    # use wandb(https://wandb.ai/)?
    IFwandb = False
    # data root
    DATA_ROOT = r'./data/'
    # save unlabel data that may be positive
    U_m_P_savepath = r"./data/U_m_P/"
    # word2vec model path
    word2vec_path = './model/model_pretrained/word2vec_pretrained.model'
    # pre_trained model path , if given None, training from scratch
    state_dict_path = './model/model_pretrained/PU-EPP_pretrained.pt'
    # Number CUDA Devices:
    gpu_number = torch.cuda.device_count()
    # DEVICE
    DEVICE = torch.device('cuda:0')

    
    
    # block layers
    layer_num = 12
    # The last dimension of the protein dataapt list --upgradable
    protein_dim = 100
    # The last dimension of the compound data
    atom_dim = 46
    # hidden dimension
    hid_dim = 128
    # norm_shape: layernorm parameter
    norm_shape = 128
    # ====================================================
    # to create trainreader
    # ====================================================
    # if use pu learning
    ifpu = True
    # if use label smoothing
    ifsmoothing = True

    # start deleting data when auc value is greater than del_threshold
    del_threshold=0.9

    quantile = 0.9


## WandB

In [42]:
if CFG.IFwandb:
    import wandb
    wandb.login()
    wandb.init(project='PU-EPP', name='finetuning')

## Parameters

### Random Seed

In [43]:
seed_everything(seed=42)

## Model

In [44]:
encoder = Encoder(CFG.protein_dim, CFG.hid_dim, CFG.norm_shape)
decoder = Decoder(CFG.atom_dim, CFG.hid_dim, CFG.norm_shape)
model = ModelCat(encoder, decoder, atom_dim=CFG.atom_dim)
model = model.to(CFG.DEVICE)
model = nn.DataParallel(model, device_ids = list(range(CFG.gpu_number)))

In [45]:
model.load_state_dict(torch.load(CFG.state_dict_path, map_location=CFG.DEVICE))

<All keys matched successfully>

In [46]:
if CFG.IFwandb:
    wandb.watch(model, log='all')

## Trainer

In [47]:
""" create trainer """
# /utils/builder.py
trainer = Trainer(model, CFG) #trainfirst=False

## Dataset & Dataloader

In [48]:
trainset = pd.read_csv(CFG.traindata_path)
testset = pd.read_csv(CFG.testdata_path)



### Dataloader

In [49]:
traindata = TrainReader(data=trainset,U_m_P_savepath=CFG.U_m_P_savepath,
                                word2vec_path=CFG.word2vec_path, ifpu=CFG.ifpu)
testdata = TestReader(data=testset, word2vec_path=CFG.word2vec_path)

''' if the pre-trained word2vec model does not include all amino acid embedding of the enzymes in your dataset, please re-train the word2vec model on your dataset'''

# train_word2vec(list(traindata['Protein'].unique()) + list(testdata['Protein'].unique()), './model/model_pretrained/word2vec_yourself.model') 

# CFG.word2vec_path = './model/model_pretrained/word2vec_yourself.model' 



' if the pre-trained word2vec model does not include all amino acid embedding of the enzymes in your dataset, please re-train the word2vec model on your dataset'

In [50]:
val_dataloader = DataLoader(testdata, batch_size=CFG.BATCH_SIZE,collate_fn=collate_fn)

## Train

In [51]:
file_model = CFG.modelsave_file_suffix
reshead = 'Epoch\tTime(sec)\tLoss_train\tAUC_dev\tPre\tRecall\tPRC_dev'
file_res = CFG.result_file_path+CFG.result_file_suffix
if os.path.exists(file_res):
    warnings.warn("----------------file name duplicate-----------------")
else:
    with open(file_res, 'w') as f:
        f.write(reshead + '\n')

/tmp/ipykernel_53091/4157713986.py:5: UserWarning: ----------------file name duplicate-----------------
  warnings.warn("----------------file name duplicate-----------------")


In [52]:
max_AUC_dev = 0
for epoch in range(CFG.EPOCHES):
    print('epoch: ' + str(epoch))
    train_dataloader = DataLoader(traindata, batch_size=CFG.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

    """Start training."""
    print('Training........................')
    torch.cuda.empty_cache()
    start = timeit.default_timer()

    loss_train, U_m_P = trainer.train(train_dataloader)
    AUC_dev, precision, recall, PRC_dev = trainer.test(val_dataloader)

    end = timeit.default_timer()
    time = end - start
    
    reslist = [epoch, time, loss_train, AUC_dev, precision, recall, PRC_dev]
    if CFG.IFwandb:
        wandb.log({'loss_train':loss_train, "AUC_dev":AUC_dev, 'pre':precision, 'recall':recall, 'prc':PRC_dev})
    trainer.save_AUCs(reslist, file_res)
    if AUC_dev > max_AUC_dev:
        trainer.save_model(model, f"{CFG.best_model_savepath}_epoch{epoch}_{AUC_dev}.pt")
        max_AUC_dev = AUC_dev
    print('\t'.join(map(str, reslist)))

    if AUC_dev > 0.85: #changed
        print(f"del {epoch}:{len(U_m_P)}")
        traindata.del_U(epoch, U_m_P)
    else:
        traindata.reset_T()
    if epoch % 5 == 0:
        trainer.save_model(model, f"{CFG.best_model_savepath}_epoch{epoch}_{AUC_dev}.pt")
    torch.cuda.empty_cache()

epoch: 0
Training........................
0	54.65413163600169	436.14746540784836	0.772134203168686	0.627906976744186	0.7297297297297297	0.7245998293785705
epoch: 1
Training........................
1	54.51413597899955	417.2627384066582	0.83690587138863	0.6666666666666666	0.7027027027027027	0.7384268687498414
epoch: 2
Training........................
2	54.73690145200089	410.62338292598724	0.8765144454799628	0.6956521739130435	0.8648648648648649	0.7857915895098386
del 2:0
epoch: 3
Training........................
3	53.396267136999086	392.7713014483452	0.8872320596458527	0.6938775510204082	0.918918918918919	0.7778610957172352
del 3:0
epoch: 4
Training........................
4	54.49071555000046	366.5054925978184	0.8974836905871388	0.68	0.918918918918919	0.7806656840749842
del 4:0
epoch: 5
Training........................
5	54.235008717998426	374.77649423480034	0.8979496738117427	0.7142857142857143	0.9459459459459459	0.7633730184470466
del 5:0
epoch: 6
Training........................
6	54.

In [53]:
max_AUC_dev

0.9384902143522833